In [393]:
import pandas as pd
print(pd.__version__)
import csv
import numpy
import os


0.25.1


In [457]:
#def normalize_RPKG_count_tab(in_tab, mc_file, lvl):
#tab = pd.read_csv(in_tab, sep="\t", header=None)

# Get GE values
mctab = pd.read_csv("5002_mc.txt", sep="\t") # read in file
ge = float(mctab.at['genome_equivalents:', 'Parameters']) # get the value for genome equivalents

ge

1090.86796555

In [439]:
# Get gene length table
lentab = pd.read_csv("megares_modified_database_v2.00_GeneLen.txt", 
                     names=['MEGAID','Class','Mechanism','Group','Gene','Len','DUM','MY'], # add column names and dummy names
                     # use both \t and |([|]) delimiter, use | to list multiple separtors
                     # Also remove the extra field of "RequiresSNPConfirmation"
                     sep="\t|[|]|RequiresSNPConfirmation", engine='python', 
                     header=None).set_index('MEGAID')

# Remove all the NaN genenrated in the middle of the df due to parsing "RequiresSNPConfirmation"
for i in lentab.index:
    if numpy.isnan(lentab.at[i,'Len']):
        lentab.at[i,'Len'] = lentab.at[i,'MY']
lentab_fin = lentab.drop(columns=['DUM', 'MY'])

# write the final gene length table out for future use
lentab_fin.to_csv("megares_modified_database_v2_GeneLen_org.tsv", mode='a', sep='\t')
lentab_fin

,Class,Mechanism,Group,Gene,Len
MEGAID,,,,,
MEG_1,Drugs,Aminoglycosides,Aminoglycoside-resistant_16S_ribosomal_subunit...,A16S,1507.0
MEG_2,Drugs,Aminoglycosides,Aminoglycoside-resistant_16S_ribosomal_subunit...,A16S,1544.0
MEG_3,Drugs,Aminoglycosides,Aminoglycoside-resistant_16S_ribosomal_subunit...,A16S,1537.0
MEG_4,Drugs,Aminoglycosides,Aminoglycoside-resistant_16S_ribosomal_subunit...,A16S,1551.0
MEG_5,Drugs,Aminoglycosides,Aminoglycoside-resistant_16S_ribosomal_subunit...,A16S,1504.0
...,...,...,...,...,...
MEG_7864,Metals,Multi-metal_resistance,Multi-metal_resistance_regulator,ZRAS,1398.0
MEG_7865,Multi-compound,Biocide_and_metal_resistance,Biocide_and_metal_resistance_protein,ZUPT,798.0
MEG_7866,Multi-compound,Biocide_and_metal_resistance,Biocide_and_metal_resistance_protein,ZUPT,774.0


In [466]:
# Get count table 
countab = pd.read_csv("5007_gene.tsv", 
                     names=['Sample','MEGID','Class','Mechanism','Group','Gene','Hits','GeneFrac','DUM','MY'], # add column names
                     sep="\t|[|]|RequiresSNPConfirmation", engine='python',
                     header=None).set_index('MEGID')
countab = countab.drop('Gene', axis=0) # Remove the first row which contains pre-parsing header

# Remove all the NaN genenrated in the middle of the df due to parsing "RequiresSNPConfirmation"
for i in countab.index:
    if numpy.isnan(countab.at[i,'Hits']):
        countab.at[i,'Hits'] = countab.at[i,'DUM']
        countab.at[i,'GeneFrac'] = countab.at[i,'MY']
countab = countab.drop(columns=['DUM','MY'])

# remove the "align" suffix after sample name
countab_fin = countab.replace('5007_align', '5007')
countab_fin


,Sample,Class,Mechanism,Group,Gene,Hits,GeneFrac
MEGID,,,,,,,
MEG_1014,5007,Drugs,Aminoglycosides,Aminoglycoside_O-phosphotransferases,APH2-DPRIME,8.0,99.8881
MEG_1016,5007,Drugs,Aminoglycosides,Aminoglycoside_O-phosphotransferases,APH2-DPRIME,6.0,86.0377
MEG_1023,5007,Drugs,Aminoglycosides,Aminoglycoside_O-phosphotransferases,APH3-DPRIME,33.0,100.0000
MEG_1039,5007,Drugs,Aminoglycosides,Aminoglycoside_O-phosphotransferases,APH3-DPRIME,8.0,80.3774
MEG_1107,5007,Biocides,Multi-biocide_resistance,Multi-biocide_resistance_regulator,ARIR,13.0,100.0000
...,...,...,...,...,...,...,...
MEG_7867,5007,Metals,Zinc_resistance,Zinc_resistance_regulator,ZUR,10.0,100.0000
MEG_7868,5007,Metals,Zinc_resistance,Zinc_resistance_regulator,ZUR,6.0,88.3721
MEG_88,5007,Drugs,Aminoglycosides,Aminoglycoside_N-acetyltransferases,AAC6-PRIME,10.0,83.7466


In [461]:
# Calculate to get the final normalized table 
normtab = countab_fin.copy()
RPKG = [] # initialize with an empty list

for i in normtab.index:
    # equation: Hits/((len in kb)*GE)
    len = lentab_fin.at[i,'Len']/1000
    norm_val = normtab.at[i,'Hits']/(len*ge) 
    RPKG.append(norm_val)
    
normtab['RPKG'] = RPKG # add a new column with RPKG

normtab


,Sample,Class,Mechanism,Group,Gene,Hits,GeneFrac,RPKG
MEGID,,,,,,,,
MEG_1014,5007,Drugs,Aminoglycosides,Aminoglycoside_O-phosphotransferases,APH2-DPRIME,8.0,99.8881,0.008203
MEG_1016,5007,Drugs,Aminoglycosides,Aminoglycoside_O-phosphotransferases,APH2-DPRIME,6.0,86.0377,0.006918
MEG_1023,5007,Drugs,Aminoglycosides,Aminoglycoside_O-phosphotransferases,APH3-DPRIME,33.0,100.0000,0.037072
MEG_1039,5007,Drugs,Aminoglycosides,Aminoglycoside_O-phosphotransferases,APH3-DPRIME,8.0,80.3774,0.009225
MEG_1107,5007,Biocides,Multi-biocide_resistance,Multi-biocide_resistance_regulator,ARIR,13.0,100.0000,0.044633
...,...,...,...,...,...,...,...,...
MEG_7867,5007,Metals,Zinc_resistance,Zinc_resistance_regulator,ZUR,10.0,100.0000,0.015915
MEG_7868,5007,Metals,Zinc_resistance,Zinc_resistance_regulator,ZUR,6.0,88.3721,0.010659
MEG_88,5007,Drugs,Aminoglycosides,Aminoglycoside_N-acetyltransferases,AAC6-PRIME,10.0,83.7466,0.006313


In [447]:
fn = os.path.basename(countfp) # get the xx.tsv file name
samid = re.search('(.+?).tsv', fn).group(1)


Class         object
Mechanism     object
Group         object
Gene          object
Len          float64
dtype: object

In [364]:
# Aggregate the per-sample file to a final count table containing multiple samples

# remove sample column and replace the RPKG column header 
normtab_fin = normtab.rename(columns={'RPKG':normtab.at[i,'Sample']}) # get sample name and rename column header
normtab_fin = normtab_fin.drop(columns=['Sample','Hits','GeneFrac']) # remove Sample, Hits, and GeneFrac columns
normtab_fin
# merge tables from different sampels by MEGAID 

,Class,Mechanism,Group,Gene,5007
MEGAID,,,,,
MEG_1014,Drugs,Aminoglycosides,Aminoglycoside_O-phosphotransferases,APH2-DPRIME,0.008203
MEG_1016,Drugs,Aminoglycosides,Aminoglycoside_O-phosphotransferases,APH2-DPRIME,0.006918
MEG_1023,Drugs,Aminoglycosides,Aminoglycoside_O-phosphotransferases,APH3-DPRIME,0.037072
MEG_1039,Drugs,Aminoglycosides,Aminoglycoside_O-phosphotransferases,APH3-DPRIME,0.009225
MEG_1107,Biocides,Multi-biocide_resistance,Multi-biocide_resistance_regulator,ARIR,0.044633
...,...,...,...,...,...
MEG_7867,Metals,Zinc_resistance,Zinc_resistance_regulator,ZUR,0.015915
MEG_7868,Metals,Zinc_resistance,Zinc_resistance_regulator,ZUR,0.010659
MEG_88,Drugs,Aminoglycosides,Aminoglycoside_N-acetyltransferases,AAC6-PRIME,0.006313


In [365]:
normtab_fin.to_csv("Sample_5007_RPKG_tab.tsv", mode='a', sep='\t')


In [404]:
fn = os.path.basename("/path/to/some/5006.tsv")
samid = re.search('(.+?).tsv', fn).group(1)
samid



'5006'